# Inspired from Tutorial2-MapMaking-QUBICPlanck-Multichromatic.Rmd written by Mathias Régnier
JCH - 17 Feb 2023

In [ ]:
from pylab import *
from pysimulators import profile

# QUBIC packages
import qubic
from qubicpack.utilities import Qubic_DataDir
from qubic.data import PATH
from qubic.io import read_map
from qubic import QubicSkySim as qss
import Acquisition as Acq

# Display packages
import healpy as hp
import matplotlib.pyplot as plt

# FG-Buster packages
import component_model as c
import mixing_matrix as mm

# General packages
import numpy as np
import pysm3
import warnings
from qubic import QubicSkySim as qss
import pysm3.units as u
from importlib import reload
from pysm3 import utils

from qubic import SpectroImLib as sp
from importlib import reload
import gc
import copy


# PyOperators packages
from pyoperators import (
    BlockColumnOperator, BlockDiagonalOperator, BlockRowOperator,
    CompositionOperator, DiagonalOperator, I, IdentityOperator,
    MPIDistributionIdentityOperator, MPI, proxy_group, PackOperator, ReshapeOperator,
    rule_manager, pcg, Operator)

from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator
warnings.filterwarnings("ignore")
%matplotlib inline

# %config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

rc('figure',figsize=(20,8))


In [ ]:
nside = 256
relative_bandwidth = 0.25
band = 220
band_planck = 217
seed = 42
noiseless = False
effective_duration = 3000
rwplanck = 1.
tol = 1e-4
noise_planck_level = 0.001
threshold = 0


npointings = 3000
Nf_TOD = 20
Nf_recon = 2
fact_sub = 5

# TOD Fabrication

In [ ]:
reload(Acq)

# Repository for dictionary
global_dir = Qubic_DataDir()
print(global_dir)
dictfilename = global_dir + 'dicts/pipeline_demo.dict'

# Read dictionary chosen
d_TOD = qubic.qubicdict.qubicDict()
d_TOD.read_from_file(dictfilename)

d_TOD['nf_recon'] = Nf_TOD
d_TOD['nf_sub'] = Nf_TOD
d_TOD['nside'] = nside
npix=12*d_TOD['nside']**2
d_TOD['RA_center'] = 0
d_TOD['DEC_center'] = -57
center = qubic.equ2gal(d_TOD['RA_center'], d_TOD['DEC_center'])
d_TOD['effective_duration'] = effective_duration
d_TOD['npointings'] = npointings
d_TOD['tol'] = 5e-4
d_TOD['filter_nu'] = band * 1e9
d_TOD['photon_noise'] = not noiseless
d_TOD['noiseless'] = noiseless
d_TOD['config'] = 'FI'
d_TOD['MultiBand'] = True
d_TOD['planck'] = True

print('*************** Noise **************')
print('Noisless:      {}'.format(d_TOD['noiseless']))
print('Photon Noise:  {}'.format(d_TOD['photon_noise']))
print('************************************')

sky_config = {'cmb':seed, 'dust':'d0'}
# sky_config = {'Dust':'d0'}
Qubic_sky = qss.Qubic_sky(sky_config, d_TOD)
mapin_TOD = Qubic_sky.get_simple_sky_map()

# QUBIC Acquisition
qubic_acquisition = Acq.QubicIntegrated(d_TOD, Nsub=Nf_TOD, Nrec=Nf_TOD)

H_TOD = qubic_acquisition.get_operator(convolution=True)
n = qubic_acquisition.get_noise()
if noiseless == True:
    n *= 0

tod = H_TOD(mapin_TOD) + n

### Free Memory
del H_TOD, qubic_acquisition, n
gc.collect()

# Reconstruction

## Build the Nrecon True maps integrated through each sub-band and convolved appropriately

In [ ]:
reload(Acq)

######## Get the unconvolved maps from pysm
d_formaps = copy.deepcopy(d_TOD)
d_formaps['nf_recon'] = Nf_recon
d_formaps['nf_sub'] = Nf_recon * fact_sub
Qubic_sky = qss.Qubic_sky(sky_config, d_formaps)
mapin = Qubic_sky.get_simple_sky_map()

######## Find the resolution for each reco sub-band. 
######## For this we artificiall say we have  Nf_recon * fact_sub in order to get all the resolutions
qubic_acquisition= Acq.QubicIntegrated(d_formaps, Nsub=Nf_recon * fact_sub, Nrec=Nf_recon)

# Convolve them to each sub-band resolution
mapin_conv = np.zeros((Nf_recon, 12*nside**2, 3))
for j in range(Nf_recon):
    print('Building True Map for reco sub-band {}'.format(j))
    for i in range(fact_sub):
        newfwhm = qubic_acquisition.allfwhm[j * fact_sub + i]
        print("     - {} Convolving input map {} to fwhm: {:5.3f} deg".format(j * fact_sub + i, i, np.degrees(newfwhm)))
        C = HealpixConvolutionGaussianOperator(fwhm=newfwhm)
        mapin_conv[j] += C(mapin[j * fact_sub + i]) / fact_sub
print(mapin_conv.shape)

mystk = 0
for mystk in range(3):
    figure()
    mini = -8
    maxi = 8
    if mystk == 0:
        mini = -300
        maxi = 300
    for k in range(Nf_recon):
        hp.gnomview(mapin_conv[k, :, mystk], rot=center, reso=25, 
                    title='True map conv - stk={} - sub={}'.format(mystk, k), sub=(1, Nf_recon, k+1), 
                    cmap='jet', min=mini, max=maxi)



## Now QUBIC and Planck acquisitions used for reconstruction

In [ ]:
def initial_guess_pcg(map, n, Nrec, npix):

    initial_guess = np.zeros((Nrec, npix, 3))
    for i in range(Nrec):
        initial_guess[i] = map + n
    
    return initial_guess


def integration(nu_min, nu_max, Nintegr, sky_config, d, fwhm_subbands = None):
    print(f'Integration from {nu_min:.2f} to {nu_max:.2f} GHz with {Nintegr} steps')
    obj = Acq.QubicIntegrated(d, Nsub=Nintegr, Nrec=Nintegr)
    if Nintegr == 1:
        allnus = np.array([np.mean([nu_min, nu_max])])
    else:
        allnus = np.linspace(nu_min, nu_max, Nintegr)
    m = obj.get_PySM_maps(sky_config, nus=allnus)
    
    if fwhm_subbands is not None:
        for i in range(Nintegr):
            C = HealpixConvolutionGaussianOperator(fwhm=fwhm_subbands[i])
            m[i] = C(m[i])
    
    return np.array([np.mean(m, axis=0)])

def initial_guess(map, nrec, fwhm = None, noisy = False):
    
    if fwhm is None:
        fwhm = [0]*nrec
    
    map_ini = np.zeros((nrec, map.shape[1], map.shape[2]))
    
    for i in range(nrec):
        #C = HealpixConvolutionGaussianOperator(fwhm = fwhm[i])
        #map_ini[i] = C(map[0])
        map_ini[i] = map[0]
        
    if nrec == 1:
        map_ini = map[0]
    
    return map_ini



In [ ]:
d = copy.deepcopy(d_formaps)

# QUBIC Acquisition and coverage
qubic_final_fwhm = list(qubic_acquisition.final_fwhm)
cov = qubic_acquisition.get_coverage()

cov = qubic_acquisition.get_coverage()

### In order to define the QUBIC and Planck regions, we use the coverage map, but first smooth it to 1 degree
C = HealpixConvolutionGaussianOperator(fwhm=np.radians(1.))
covconv = C(cov)

seenpix = (cov > 0) & ((covconv/covconv.max()) > threshold)
maskinit = np.array(seenpix, dtype=int)
maskinit[np.where(seenpix == False)[0]] = 1e10

### Now we can also apodize the mask
mask = maskinit.copy()

rc('figure',figsize=(20,8))
hp.gnomview(cov/cov.max(), min=0, max=1, rot=center, reso=25, cmap='jet', sub=(1,4,1), title='Coverage Map')
hp.gnomview(maskinit, rot=center, reso=25, cmap='jet', sub=(1,4,2), title='Planck Mask Init')


In [ ]:
# Planck Acquisition
planck_acquisition = Acq.PlanckAcquisition(band_planck, qubic_acquisition.scene)
m_planck = integration(band-35, band+35, qubic_acquisition.Nsub, sky_config, d, fwhm_subbands=qubic_acquisition.allfwhm)
map_planck_ini = initial_guess(m_planck+np.array([planck_acquisition.get_noise()*noise_planck_level]), Nf_recon, fwhm = qubic_acquisition.final_fwhm)

# Joint Acquisition
qubicplanck_acquisition = Acq.QubicPlanckMultiBandAcquisition(qubic_acquisition, planck_acquisition)

# Free som memory
#del qubic_acquisition, planck_acquisition
#gc.collect()

H = qubicplanck_acquisition.get_operator(convolution=False, convolve_to_max=False)

invntt = qubicplanck_acquisition.get_invntt_operator(weight_planck=1, beam_correction=qubic_final_fwhm)

invntt

In [ ]:
### Replacing fake QUBIC TOD with real one
n = qubicplanck_acquisition.get_noise()
if noiseless:
    n *= 0
# PCH: Pourquoi multiplier par noise_planck_level ?
mytod = H(mapin_conv) + n * noise_planck_level
nelt = len(np.ravel(tod))
mytod[:nelt] = np.ravel(tod)


### Solve PCG
U = (
    ReshapeOperator((Nf_recon * sum(seenpix) * 3), (Nf_recon, sum(seenpix), 3)) *
    PackOperator(np.broadcast_to(seenpix[None, :, None], (Nf_recon, seenpix.size, 3)).copy())
).T

with rule_manager(none=True):
    A = U.T * H.T * invntt * H * U
x_planck = map_planck_ini * (1 - seenpix[None, :, None])
b = U.T ( H.T * invntt * (mytod - H(x_planck)) )

def get_preconditioner(cov):
    if cov is not None:
        cov_inv = 1 / cov
        cov_inv[np.isinf(cov_inv)] = 0.
        preconditioner = DiagonalOperator(cov_inv, broadcast='rightward')
    else:
        preconditioner = None
    return preconditioner

M = Acq.get_preconditioner(np.ones(12*d['nside']**2))

print('A', A.shapein, A.shapeout)
print('b', b.shape)
print('map_planck_ini', map_planck_ini.shape)
print('M', M.shapein, M.shapeout)

solution_qubic_planck = pcg(A, b, x0=U.T(map_planck_ini), M=M, tol=tol*0+1e-8, disp=True, maxiter=100)
x_qubic = U(solution_qubic_planck['x'])
solution_qubic_planck['x'] = x_planck + x_qubic

del H, M, A, b, n, mytod, invntt#, qubicplanck_acquisition
gc.collect()

In [ ]:
from qubic import fibtools as ft
def display_maps(inputs, outputs, istk, display=True, rot=None, res=None, min=300, max=300, minr=-300, maxr=300, 
                 extra_string='', addnum=False, inside=None, cov=None):
    
    r = inputs[:, :, istk] - outputs[:, :, istk]
    Nf = outputs.shape[0]
    
    if cov is None:
        nx = 3
    else:
        nx = 4
        covnorm = cov/np.max(cov)
        uvpix = np.array(hp.pix2vec(hp.npix2nside(len(cov)), np.arange(len(cov))))
        uvcenter = np.array(hp.pix2vec(hp.npix2nside(len(cov)), argmax(cov)))
        dcenter = np.degrees(np.arccos(np.dot(uvpix.T, uvcenter)))
    
    all_rms_in = np.zeros(Nf)
    all_rms_out = np.zeros(Nf)
    
    stk=['I', 'Q', 'U']
    k=1
    for i in range(Nf): # Nf
        if addnum:
            addstr = '{}'.format(i)
        else:
            addstr=''
        if inside is not None:
            rms_in = np.std(r[i][inside])
            rms_out = np.std(r[i][~inside])
            all_rms_in[i] = rms_in
            all_rms_out[i] = rms_out
            addin = '\n $\sigma_{Qubic}$='+'{0:5.2g}'.format(rms_in)
        else:
            addin = ''
        hp.gnomview(inputs[i, :, istk], rot=rot, reso=res, cmap='jet', min=min, max=max, sub=(Nf, nx, k), title='Input'+ extra_string+addstr)
        k+=1
        hp.gnomview(outputs[i, :, istk], rot=rot, reso=res, cmap='jet', min=min, max=max, sub=(Nf, nx, k), title='Output'+ extra_string+addstr)
        k+=1
        hp.gnomview(r[i], rot=rot, reso=res, cmap='jet', min=minr, max=maxr, sub=(Nf, nx, k), title='Residual'+ extra_string+addstr+addin)
        k+=1
        if nx==4:
            x,y,sx,sy,_ = ft.profile(dcenter, r[i], nbins=40, rng=[0,40], plot=False)
            subplot(2, nx, 4)
            errorbar(x, y,yerr=sy, fmt='o')
            xlabel('Radius [deg]')
            ylabel('Mean')
            ylim(-maxr, maxr)
            if istk==0:
                ylim(-maxr*5, maxr*5)
            subplot(2, nx, 8)
            plot(x, sy,'o-')
            xlabel('Radius [deg]')
            ylabel('RMS Residuals')
            ylim(0,maxr)
            if istk==0:
                ylim(0, maxr*5)
            k+=1
        
    return all_rms_in, all_rms_out


rc('figure',figsize=(20,8))
figure()
hp.gnomview(cov/cov.max(), min=0, max=1, rot=center, reso=25, cmap='jet', sub=(2,4,1), title='Coverage Map')
hp.gnomview(mask, rot=center, reso=25, cmap='jet', sub=(2,4,2), title='Planck Mask')
hp.gnomview(seenpix, rot=center, reso=25, cmap='jet', sub=(2,4,3), title='Seenpix')
hp.gnomview(mask, rot=center, reso=25, cmap='jet', sub=(2,4,4), title='Planck Mask')
#tight_layout()
figure()
rc('figure',figsize=(20,8))
### mapin_conv contains QUBIC frequency maps. In the Planck region, we need to replace them by the Planck map values.
myrefmap = mapin_conv.copy()
myrefmap[:,~seenpix,:] = m_planck[0,~seenpix,:]

all_rms_in = np.zeros((Nf_recon, 3))
all_rms_out = np.zeros((Nf_recon, 3))
inside = (cov/np.max(cov)) >= threshold
stk = ['I', 'Q', 'U']
for i in range(3):
    #fig = plt.figure(figsize=(10, 10))
    fig = plt.figure()
    mm = 20
    mmr = mm/10
    if i==0:
        mm=300
        mmr = 10
    rin, rout = display_maps(myrefmap, solution_qubic_planck['x'], istk=i, display=True, rot=center, res=25, 
                 min=-mm, max=mm, minr=-mmr, maxr=mmr, extra_string=' {}'.format(stk[i]), addnum=True, inside=inside, cov=cov)
    all_rms_in[:,i] = rin
    all_rms_out[:,i] = rout
    fig.suptitle('Stokes {} - Thr = {}'.format(stk[i], threshold), fontsize=20, y=1.1)
    plt.tight_layout()
    plt.show()
    
print('RMS In:')
#print(all_rms_in)
print('Mean')
print(np.mean(all_rms_in, axis=0))
print()
print('RMS Out:')
#print(all_rms_out)
print('Mean')
print(np.mean(all_rms_out, axis=0))


TODO:
- essayer de reduire la zone QUBIC et de remplacer par planck jusqu'à cov=0.1 par exemple. Peut etre mieux...

In [ ]:
istk = 0
hp.gnomview(solution_qubic_planck['x'][1,:,istk] - solution_qubic_planck['x'][0,:,istk], rot=center, reso=25, cmap='jet', min=-1, max=1)

In [ ]:
solution_qubic_planck['x'][1,seenpix,istk] - solution_qubic_planck['x'][0,seenpix,istk]